<a id="item31"></a>


## Regression with Keras assignment- This part uses three hidden layers with 10 nodes each compared to only one hidded layer in part B of the assignment. This notebook uses the standard scaling approach (subtracting the mean and dividing by the standard deviation)


In [4]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==2.0.2
#!pip install pandas==2.2.2
#!pip install tensorflow_cpu==2.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 88.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 79.7 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 64.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 23.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

## Importing the required libraries

In [1]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import mean_squared_error

## Read the data

In [5]:
filepath='concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Separating the predictors and the target

In [8]:
concrete_data_columns = concrete_data.columns

In [9]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


## Normalizing the data

In [10]:
predictors_normalized = (predictors - predictors.mean()) / predictors.std()
predictors_normalized.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


## Train-test splitting of the data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(predictors_normalized, target, test_size=0.30, random_state=42)

## Building a neural network with keras using 3 hidden layers of 10 nodes,  ReLu activation function, adam optimizer and the mean squared error  as the loss function.

In [12]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(predictors.shape[1],))) # input layer with number of predictors
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Model initialization

In [13]:
model = regression_model()

## Model training by fitting the model instance on the training data

In [14]:
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
16/16 - 6s - loss: 1622.1614 - val_loss: 1493.8608 - 6s/epoch - 356ms/step
Epoch 2/50
16/16 - 0s - loss: 1609.4460 - val_loss: 1480.6742 - 147ms/epoch - 9ms/step
Epoch 3/50
16/16 - 0s - loss: 1593.8654 - val_loss: 1463.1486 - 152ms/epoch - 10ms/step
Epoch 4/50
16/16 - 0s - loss: 1572.7368 - val_loss: 1437.3854 - 138ms/epoch - 9ms/step
Epoch 5/50
16/16 - 0s - loss: 1542.1124 - val_loss: 1398.8256 - 132ms/epoch - 8ms/step
Epoch 6/50
16/16 - 0s - loss: 1496.4009 - val_loss: 1342.6445 - 137ms/epoch - 9ms/step
Epoch 7/50
16/16 - 0s - loss: 1428.4603 - val_loss: 1261.1005 - 154ms/epoch - 10ms/step
Epoch 8/50
16/16 - 0s - loss: 1332.3187 - val_loss: 1145.8090 - 156ms/epoch - 10ms/step
Epoch 9/50
16/16 - 0s - loss: 1202.0944 - val_loss: 996.1281 - 130ms/epoch - 8ms/step
Epoch 10/50
16/16 - 0s - loss: 1033.4093 - val_loss: 823.5203 - 134ms/epoch - 8ms/step
Epoch 11/50
16/16 - 0s - loss: 850.9985 - val_loss: 641.9954 - 125ms/epoch - 8ms/step
Epoch 12/50
16/16 - 0s - loss: 663.7665 - v

## Calculating model predictions over the test data using model.predict()

In [15]:
predicted_strength = model.predict(X_test)

10/10 [==============================] - 0s 2ms/step


In [16]:
#predicted_strength

## Calculating mean squared error from mean_squared_error of scikit learn

In [17]:
mse = mean_squared_error(y_test, predicted_strength)

In [18]:
print('Mean squared error on the test data is: ', mse)

Mean squared error on the test data is:  135.7001428897702


## Repeating, splitting and model trainining 50 times and calculating the mse and standard deviation of MSE

In [19]:
mse_50 = []
model = regression_model()
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)
    predictions = model.predict(X_test)
    mse_50.append(mean_squared_error(y_test, predictions))

Epoch 1/50
16/16 - 1s - loss: 446.6266 - val_loss: 282.0598 - 710ms/epoch - 44ms/step
Epoch 2/50
16/16 - 0s - loss: 242.0246 - val_loss: 245.6053 - 116ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 216.6311 - val_loss: 227.9126 - 134ms/epoch - 8ms/step
Epoch 4/50
16/16 - 0s - loss: 191.3851 - val_loss: 198.2717 - 143ms/epoch - 9ms/step
Epoch 5/50
16/16 - 0s - loss: 164.1806 - val_loss: 162.2516 - 149ms/epoch - 9ms/step
Epoch 6/50
16/16 - 0s - loss: 139.4570 - val_loss: 141.7434 - 144ms/epoch - 9ms/step
Epoch 7/50
16/16 - 0s - loss: 126.7838 - val_loss: 130.9042 - 154ms/epoch - 10ms/step
Epoch 8/50
16/16 - 0s - loss: 117.8943 - val_loss: 122.5220 - 130ms/epoch - 8ms/step
Epoch 9/50
16/16 - 0s - loss: 112.3083 - val_loss: 115.5999 - 182ms/epoch - 11ms/step
Epoch 10/50
16/16 - 0s - loss: 105.4044 - val_loss: 111.6906 - 187ms/epoch - 12ms/step
Epoch 11/50
16/16 - 0s - loss: 99.6851 - val_loss: 107.5008 - 143ms/epoch - 9ms/step
Epoch 12/50
16/16 - 0s - loss: 95.7574 - val_loss: 109.6304 

Epoch 48/50
16/16 - 0s - loss: 57.4272 - val_loss: 68.8482 - 108ms/epoch - 7ms/step
Epoch 49/50
16/16 - 0s - loss: 57.5660 - val_loss: 75.6263 - 109ms/epoch - 7ms/step
Epoch 50/50
16/16 - 0s - loss: 59.4355 - val_loss: 68.9522 - 118ms/epoch - 7ms/step
10/10 [==============================] - 0s 2ms/step
Epoch 1/50
16/16 - 0s - loss: 65.5075 - val_loss: 54.7658 - 164ms/epoch - 10ms/step
Epoch 2/50
16/16 - 0s - loss: 65.7724 - val_loss: 55.3716 - 119ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 66.2005 - val_loss: 61.0589 - 108ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 66.6971 - val_loss: 56.4779 - 119ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 65.9293 - val_loss: 58.7384 - 124ms/epoch - 8ms/step
Epoch 6/50
16/16 - 0s - loss: 69.7871 - val_loss: 56.0210 - 122ms/epoch - 8ms/step
Epoch 7/50
16/16 - 0s - loss: 64.3396 - val_loss: 56.9541 - 121ms/epoch - 8ms/step
Epoch 8/50
16/16 - 0s - loss: 65.9819 - val_loss: 61.9752 - 133ms/epoch - 8ms/step
Epoch 9/50
16/16 - 0s - loss: 

Epoch 45/50
16/16 - 0s - loss: 50.3606 - val_loss: 64.5771 - 111ms/epoch - 7ms/step
Epoch 46/50
16/16 - 0s - loss: 51.6193 - val_loss: 66.6462 - 116ms/epoch - 7ms/step
Epoch 47/50
16/16 - 0s - loss: 50.2194 - val_loss: 66.9401 - 112ms/epoch - 7ms/step
Epoch 48/50
16/16 - 0s - loss: 50.2926 - val_loss: 67.4293 - 117ms/epoch - 7ms/step
Epoch 49/50
16/16 - 0s - loss: 49.1923 - val_loss: 64.1461 - 116ms/epoch - 7ms/step
Epoch 50/50
16/16 - 0s - loss: 49.2833 - val_loss: 63.6086 - 127ms/epoch - 8ms/step
10/10 [==============================] - 0s 3ms/step
Epoch 1/50
16/16 - 0s - loss: 54.3314 - val_loss: 49.3337 - 172ms/epoch - 11ms/step
Epoch 2/50
16/16 - 0s - loss: 54.9282 - val_loss: 58.3399 - 122ms/epoch - 8ms/step
Epoch 3/50
16/16 - 0s - loss: 54.4309 - val_loss: 49.5837 - 114ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 51.1223 - val_loss: 48.7364 - 117ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 51.1054 - val_loss: 49.8413 - 125ms/epoch - 8ms/step
Epoch 6/50
16/16 - 0s - los

Epoch 42/50
16/16 - 0s - loss: 39.4685 - val_loss: 55.1256 - 135ms/epoch - 8ms/step
Epoch 43/50
16/16 - 0s - loss: 39.1785 - val_loss: 53.8014 - 109ms/epoch - 7ms/step
Epoch 44/50
16/16 - 0s - loss: 39.2689 - val_loss: 54.9714 - 113ms/epoch - 7ms/step
Epoch 45/50
16/16 - 0s - loss: 39.2487 - val_loss: 53.0785 - 120ms/epoch - 7ms/step
Epoch 46/50
16/16 - 0s - loss: 39.3863 - val_loss: 54.2258 - 109ms/epoch - 7ms/step
Epoch 47/50
16/16 - 0s - loss: 39.2130 - val_loss: 53.7618 - 115ms/epoch - 7ms/step
Epoch 48/50
16/16 - 0s - loss: 39.0295 - val_loss: 53.8635 - 128ms/epoch - 8ms/step
Epoch 49/50
16/16 - 0s - loss: 40.0782 - val_loss: 54.7012 - 127ms/epoch - 8ms/step
Epoch 50/50
16/16 - 0s - loss: 39.6104 - val_loss: 58.6790 - 119ms/epoch - 7ms/step
10/10 [==============================] - 0s 2ms/step
Epoch 1/50
16/16 - 0s - loss: 48.3815 - val_loss: 46.8050 - 168ms/epoch - 10ms/step
Epoch 2/50
16/16 - 0s - loss: 47.2462 - val_loss: 44.7018 - 123ms/epoch - 8ms/step
Epoch 3/50
16/16 - 0s - 

Epoch 39/50
16/16 - 0s - loss: 38.9372 - val_loss: 68.0836 - 134ms/epoch - 8ms/step
Epoch 40/50
16/16 - 0s - loss: 39.5799 - val_loss: 57.5591 - 122ms/epoch - 8ms/step
Epoch 41/50
16/16 - 0s - loss: 37.3521 - val_loss: 57.5153 - 127ms/epoch - 8ms/step
Epoch 42/50
16/16 - 0s - loss: 38.1567 - val_loss: 55.5695 - 116ms/epoch - 7ms/step
Epoch 43/50
16/16 - 0s - loss: 37.7344 - val_loss: 55.1273 - 112ms/epoch - 7ms/step
Epoch 44/50
16/16 - 0s - loss: 38.5579 - val_loss: 54.5362 - 109ms/epoch - 7ms/step
Epoch 45/50
16/16 - 0s - loss: 40.0759 - val_loss: 63.2597 - 109ms/epoch - 7ms/step
Epoch 46/50
16/16 - 0s - loss: 37.7532 - val_loss: 55.2796 - 111ms/epoch - 7ms/step
Epoch 47/50
16/16 - 0s - loss: 38.2383 - val_loss: 55.2673 - 118ms/epoch - 7ms/step
Epoch 48/50
16/16 - 0s - loss: 38.5259 - val_loss: 57.8538 - 128ms/epoch - 8ms/step
Epoch 49/50
16/16 - 0s - loss: 38.2073 - val_loss: 59.9278 - 120ms/epoch - 7ms/step
Epoch 50/50
16/16 - 0s - loss: 38.3186 - val_loss: 55.7764 - 134ms/epoch - 8

...

Epoch 38/50
16/16 - 0s - loss: 31.7169 - val_loss: 37.0624 - 117ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 32.2558 - val_loss: 37.0399 - 134ms/epoch - 8ms/step
Epoch 40/50
16/16 - 0s - loss: 32.4007 - val_loss: 37.4796 - 123ms/epoch - 8ms/step
Epoch 41/50
16/16 - 0s - loss: 32.3511 - val_loss: 36.5091 - 135ms/epoch - 8ms/step
Epoch 42/50
16/16 - 0s - loss: 31.6253 - val_loss: 37.1572 - 122ms/epoch - 8ms/step
Epoch 43/50
16/16 - 0s - loss: 32.5534 - val_loss: 36.4643 - 116ms/epoch - 7ms/step
Epoch 44/50
16/16 - 0s - loss: 31.6196 - val_loss: 38.4412 - 116ms/epoch - 7ms/step
Epoch 45/50
16/16 - 0s - loss: 31.4189 - val_loss: 38.4462 - 110ms/epoch - 7ms/step
Epoch 46/50
16/16 - 0s - loss: 33.0705 - val_loss: 37.4968 - 119ms/epoch - 7ms/step
Epoch 47/50
16/16 - 0s - loss: 31.7642 - val_loss: 36.8716 - 122ms/epoch - 8ms/step
Epoch 48/50
16/16 - 0s - loss: 31.5135 - val_loss: 36.9990 - 109ms/epoch - 7ms/step
Epoch 49/50
16/16 - 0s - loss: 32.1773 - val_loss: 38.3960 - 119ms/epoch - 7

Epoch 35/50
16/16 - 0s - loss: 30.7815 - val_loss: 33.6029 - 119ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 31.2032 - val_loss: 33.6655 - 116ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 31.4597 - val_loss: 33.8752 - 124ms/epoch - 8ms/step
Epoch 38/50
16/16 - 0s - loss: 31.1858 - val_loss: 32.7579 - 116ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 30.2488 - val_loss: 32.4945 - 124ms/epoch - 8ms/step
Epoch 40/50
16/16 - 0s - loss: 30.5861 - val_loss: 33.0579 - 109ms/epoch - 7ms/step
Epoch 41/50
16/16 - 0s - loss: 30.0638 - val_loss: 34.3622 - 138ms/epoch - 9ms/step
Epoch 42/50
16/16 - 0s - loss: 30.1272 - val_loss: 33.3824 - 128ms/epoch - 8ms/step
Epoch 43/50
16/16 - 0s - loss: 30.5363 - val_loss: 33.1120 - 117ms/epoch - 7ms/step
Epoch 44/50
16/16 - 0s - loss: 30.2553 - val_loss: 35.3707 - 121ms/epoch - 8ms/step
Epoch 45/50
16/16 - 0s - loss: 31.0093 - val_loss: 33.0985 - 128ms/epoch - 8ms/step
Epoch 46/50
16/16 - 0s - loss: 31.9559 - val_loss: 33.0480 - 114ms/epoch - 7

Epoch 32/50
16/16 - 0s - loss: 29.0047 - val_loss: 30.7949 - 113ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 29.3630 - val_loss: 32.2571 - 141ms/epoch - 9ms/step
Epoch 34/50
16/16 - 0s - loss: 31.3233 - val_loss: 33.7082 - 114ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 30.9207 - val_loss: 30.7830 - 113ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 29.0193 - val_loss: 29.9255 - 116ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 29.5489 - val_loss: 30.1741 - 132ms/epoch - 8ms/step
Epoch 38/50
16/16 - 0s - loss: 29.3014 - val_loss: 31.0296 - 121ms/epoch - 8ms/step
Epoch 39/50
16/16 - 0s - loss: 31.9985 - val_loss: 32.5766 - 134ms/epoch - 8ms/step
Epoch 40/50
16/16 - 0s - loss: 29.2084 - val_loss: 30.4352 - 118ms/epoch - 7ms/step
Epoch 41/50
16/16 - 0s - loss: 29.6270 - val_loss: 31.6394 - 125ms/epoch - 8ms/step
Epoch 42/50
16/16 - 0s - loss: 29.0304 - val_loss: 29.9598 - 130ms/epoch - 8ms/step
Epoch 43/50
16/16 - 0s - loss: 29.5859 - val_loss: 30.1607 - 118ms/epoch - 7

Epoch 29/50
16/16 - 0s - loss: 29.1818 - val_loss: 30.7389 - 126ms/epoch - 8ms/step
Epoch 30/50
16/16 - 0s - loss: 31.3971 - val_loss: 29.3578 - 206ms/epoch - 13ms/step
Epoch 31/50
16/16 - 0s - loss: 29.8720 - val_loss: 29.2296 - 148ms/epoch - 9ms/step
Epoch 32/50
16/16 - 0s - loss: 30.1315 - val_loss: 27.9649 - 114ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 29.3244 - val_loss: 31.2311 - 122ms/epoch - 8ms/step
Epoch 34/50
16/16 - 0s - loss: 29.5093 - val_loss: 28.9402 - 131ms/epoch - 8ms/step
Epoch 35/50
16/16 - 0s - loss: 30.3471 - val_loss: 29.0783 - 120ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 29.6104 - val_loss: 28.5291 - 114ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 28.8619 - val_loss: 29.5809 - 123ms/epoch - 8ms/step
Epoch 38/50
16/16 - 0s - loss: 29.5194 - val_loss: 29.1593 - 129ms/epoch - 8ms/step
Epoch 39/50
16/16 - 0s - loss: 29.8966 - val_loss: 29.0285 - 133ms/epoch - 8ms/step
Epoch 40/50
16/16 - 0s - loss: 28.9776 - val_loss: 29.1236 - 130ms/epoch - 

Epoch 26/50
16/16 - 0s - loss: 30.8226 - val_loss: 31.4852 - 133ms/epoch - 8ms/step
Epoch 27/50
16/16 - 0s - loss: 30.3188 - val_loss: 32.4605 - 131ms/epoch - 8ms/step
Epoch 28/50
16/16 - 0s - loss: 31.5545 - val_loss: 32.4120 - 122ms/epoch - 8ms/step
Epoch 29/50
16/16 - 0s - loss: 28.9242 - val_loss: 30.7845 - 137ms/epoch - 9ms/step
Epoch 30/50
16/16 - 0s - loss: 29.6194 - val_loss: 32.0962 - 119ms/epoch - 7ms/step
Epoch 31/50
16/16 - 0s - loss: 30.4215 - val_loss: 32.9788 - 135ms/epoch - 8ms/step
Epoch 32/50
16/16 - 0s - loss: 29.0033 - val_loss: 32.1048 - 112ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 29.4114 - val_loss: 32.6463 - 109ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 29.4732 - val_loss: 31.8619 - 112ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 29.9796 - val_loss: 31.8143 - 112ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 29.3359 - val_loss: 35.4909 - 108ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 29.5840 - val_loss: 36.1792 - 117ms/epoch - 7

## Calculating mean and standard deviation of the MSEs using numpy 

In [20]:
mse_arr = np.array(mse_50)
mean_mse = np.mean(mse_arr)
std_mse = np.std(mse_arr)

In [21]:
print('The mean MSE after training the model 50 times is: ',mean_mse, ' and the standard deviation is: ', std_mse)

The mean MSE after training the model 50 times is:  42.23996217769971  and the standard deviation is:  9.959421353902961


Increasing the number of hidden layers as compared to part B (with  one hidden layer) has improved the model performance both in terms of Mean MSE as well as standard deviation. This Model has yielded better performance than all, PartA (Using unscaled data with single hidden layer with 50 training epochs), PartB (Using scaled data, one hidden layer, and 50 training epochs), and Part C (Using scaled data, with single hidden layers but 100 training epochs). 
The current model used 50 training epochs over the scaled data with three hidden layers in the Keras sequential model

## PartA: Use of unscaled data, one hidden layer with 10 nodes, Relu activation, Adam optimizer and MSE loss function, 50 training epochs:

Performance- The mean MSE after training the model 50 times is:  58.269382589504986  and the standard deviation is:  18.803301390418138

## PartB: Use of scaled data, one hidden layer with 10 nodes, Relu activation, Adam optimizer and MSE loss function, 50 training epochs.

PartB-Scaling by subtracting Mean and dividing by the standard deviation (standard scaling).
Performance- The mean MSE after training the model 50 times is:  92.61864332056368  and the standard deviation is:  85.06794677610809

PartB-Scaling by MinMax scaler.
Performance- The mean MSE after training the model 50 times after the data is scaled using minmax scaler is:  74.74712459720027  and the standard deviation is:  45.93906271045597

PartB-Scaling by Robust scaler.
Performance- The mean MSE after training the model 50 times after the data is scaled using robust scaler is:  53.39723753448831  and the standard deviation is:  77.20912119999225

## PartC: Use of scaled data, one hidden layer with 10 nodes, Relu activation, Adam optimizer and MSE loss function, 100 training epochs.
Performance-The mean MSE after training the model 50 times with 100 epochs:  83.85968388580191  and the standard deviation is:  93.42974445371604

## PartD: Use of scaled data, Thre hidden layerers with 10 nodes, Relu activation, Adam optimizer and MSE loss function, 50 training epochs.
Performance- The mean MSE after training the model 50 times is:  42.23996217769971  and the standard deviation is:  9.959421353902961